# 联邦学习

## 导入数据

In [4]:
import numpy as np
from collections import Counter
import random
import sys
import codecs
np.random.seed(12345)
with codecs.open('spam.txt','r', encoding='utf-8', errors='ignore') as f:
    raw = f.readlines()
vocab, spam, ham = (set(['unk']), list(), list())
for row in raw:
    spam.append(set(row[:-2].split(' ')))
    for word in spam[-1]:
        vocab.add(word)

with codecs.open('ham.txt', 'r', encoding='utf-8', errors='ignore') as f:
    raw = f.readlines()
    
for row in raw:
    ham.append((set(row[:-2].split(' '))))
    for word in ham[-1]:
        vocab.add(word)
        
vocab, word2index = (list(vocab), {})
for i, w in enumerate(vocab):
    word2index[w] = i
    
def to_indices(input, length=500):
    indices = list()
    for line in input:
        if len(line) < length:
            line = list(line) + ['unk'] * (length - len(line))
            idxs = list()
            for word in line:
                idxs.append(word2index[word])
            indices.append(idxs)
    return indices

In [5]:
import numpy as np

class Tensor (object):
    
    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        
        self.data = np.array(data)
        self.autograd = autograd
        self.grad = None

        if(id is None):
            self.id = np.random.randint(0,1000000000)
        else:
            self.id = id
        
        self.creators = creators
        self.creation_op = creation_op
        self.children = {}
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1

    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt != 0):
                return False
        return True 
        
    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
 
            if(grad is None):
                grad = Tensor(np.ones_like(self.data))

            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    return
                    print(self.id)
                    print(self.creation_op)
                    print(len(self.creators))
                    for c in self.creators:
                        print(c.creation_op)
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad
            
            # grads must not have grads of their own
            assert grad.autograd == False
            
            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if(self.creators is not None and 
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if(self.creation_op == "add"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                    
                if(self.creation_op == "sub"):
                    self.creators[0].backward(Tensor(self.grad.data), self)
                    self.creators[1].backward(Tensor(self.grad.__neg__().data), self)

                if(self.creation_op == "mul"):
                    new = self.grad * self.creators[1]
                    self.creators[0].backward(new , self)
                    new = self.grad * self.creators[0]
                    self.creators[1].backward(new, self)                    
                    
                if(self.creation_op == "mm"):
                    c0 = self.creators[0]
                    c1 = self.creators[1]
                    new = self.grad.mm(c1.transpose())
                    c0.backward(new)
                    new = self.grad.transpose().mm(c0).transpose()
                    c1.backward(new)
                    
                if(self.creation_op == "transpose"):
                    self.creators[0].backward(self.grad.transpose())

                if("sum" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.expand(dim,
                                                               self.creators[0].data.shape[dim]))

                if("expand" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.sum(dim))
                    
                if(self.creation_op == "neg"):
                    self.creators[0].backward(self.grad.__neg__())
                    
                if(self.creation_op == "sigmoid"):
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (self * (ones - self)))
                
                if(self.creation_op == "tanh"):
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (ones - (self * self)))
                
                if(self.creation_op == "index_select"):
                    new_grad = np.zeros_like(self.creators[0].data)
                    indices_ = self.index_select_indices.data.flatten()
                    grad_ = grad.data.reshape(len(indices_), -1)
                    for i in range(len(indices_)):
                        new_grad[indices_[i]] += grad_[i]
                    self.creators[0].backward(Tensor(new_grad))
                    
                if(self.creation_op == "cross_entropy"):
                    dx = self.softmax_output - self.target_dist
                    self.creators[0].backward(Tensor(dx))
                    
    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="add")
        return Tensor(self.data + other.data)

    def __neg__(self):
        if(self.autograd):
            return Tensor(self.data * -1,
                          autograd=True,
                          creators=[self],
                          creation_op="neg")
        return Tensor(self.data * -1)
    
    def __sub__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data - other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="sub")
        return Tensor(self.data - other.data)
    
    def __mul__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data * other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="mul")
        return Tensor(self.data * other.data)    

    def sum(self, dim):
        if(self.autograd):
            return Tensor(self.data.sum(dim),
                          autograd=True,
                          creators=[self],
                          creation_op="sum_"+str(dim))
        return Tensor(self.data.sum(dim))
    
    def expand(self, dim,copies):

        trans_cmd = list(range(0,len(self.data.shape)))
        trans_cmd.insert(dim,len(self.data.shape))
        new_data = self.data.repeat(copies).reshape(list(self.data.shape) + [copies]).transpose(trans_cmd)
        
        if(self.autograd):
            return Tensor(new_data,
                          autograd=True,
                          creators=[self],
                          creation_op="expand_"+str(dim))
        return Tensor(new_data)
    
    def transpose(self):
        if(self.autograd):
            return Tensor(self.data.transpose(),
                          autograd=True,
                          creators=[self],
                          creation_op="transpose")
        
        return Tensor(self.data.transpose())
    
    def mm(self, x):
        if(self.autograd):
            return Tensor(self.data.dot(x.data),
                          autograd=True,
                          creators=[self,x],
                          creation_op="mm")
        return Tensor(self.data.dot(x.data))
    
    def sigmoid(self):
        if(self.autograd):
            return Tensor(1 / (1 + np.exp(-self.data)),
                          autograd=True,
                          creators=[self],
                          creation_op="sigmoid")
        return Tensor(1 / (1 + np.exp(-self.data)))

    def tanh(self):
        if(self.autograd):
            return Tensor(np.tanh(self.data),
                          autograd=True,
                          creators=[self],
                          creation_op="tanh")
        return Tensor(np.tanh(self.data))
    
    def index_select(self, indices):

        if(self.autograd):
            new = Tensor(self.data[indices.data],
                         autograd=True,
                         creators=[self],
                         creation_op="index_select")
            new.index_select_indices = indices
            return new
        return Tensor(self.data[indices.data])
    
    def softmax(self):
        temp = np.exp(self.data)
        softmax_output = temp / np.sum(temp,
                                       axis=len(self.data.shape)-1,
                                       keepdims=True)
        return softmax_output
    
    def cross_entropy(self, target_indices):

        temp = np.exp(self.data)
        softmax_output = temp / np.sum(temp,
                                       axis=len(self.data.shape)-1,
                                       keepdims=True)
        
        t = target_indices.data.flatten()
        p = softmax_output.reshape(len(t),-1)
        target_dist = np.eye(p.shape[1])[t]
        loss = -(np.log(p) * (target_dist)).sum(1).mean()
    
        if(self.autograd):
            out = Tensor(loss,
                         autograd=True,
                         creators=[self],
                         creation_op="cross_entropy")
            out.softmax_output = softmax_output
            out.target_dist = target_dist
            return out

        return Tensor(loss)
        
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())  

class Layer(object):
    
    def __init__(self):
        self.parameters = list()
        
    def get_parameters(self):
        return self.parameters

    
class SGD(object):
    
    def __init__(self, parameters, alpha=0.1):
        self.parameters = parameters
        self.alpha = alpha
    
    def zero(self):
        for p in self.parameters:
            p.grad.data *= 0
        
    def step(self, zero=True):
        
        for p in self.parameters:
            
            p.data -= p.grad.data * self.alpha
            
            if(zero):
                p.grad.data *= 0


class Linear(Layer):

    def __init__(self, n_inputs, n_outputs, bias=True):
        super().__init__()
        
        self.use_bias = bias
        
        W = np.random.randn(n_inputs, n_outputs) * np.sqrt(2.0/(n_inputs))
        self.weight = Tensor(W, autograd=True)
        if(self.use_bias):
            self.bias = Tensor(np.zeros(n_outputs), autograd=True)
        
        self.parameters.append(self.weight)
        
        if(self.use_bias):        
            self.parameters.append(self.bias)

    def forward(self, input):
        if(self.use_bias):
            return input.mm(self.weight)+self.bias.expand(0,len(input.data))
        return input.mm(self.weight)


class Sequential(Layer):
    
    def __init__(self, layers=list()):
        super().__init__()
        
        self.layers = layers
    
    def add(self, layer):
        self.layers.append(layer)
        
    def forward(self, input):
        for layer in self.layers:
            input = layer.forward(input)
        return input
    
    def get_parameters(self):
        params = list()
        for l in self.layers:
            params += l.get_parameters()
        return params


class Embedding(Layer):
    
    def __init__(self, vocab_size, dim):
        super().__init__()
        
        self.vocab_size = vocab_size
        self.dim = dim
        
        # this random initialiation style is just a convention from word2vec
        self.weight = Tensor((np.random.rand(vocab_size, dim) - 0.5) / dim, autograd=True)
        
        self.parameters.append(self.weight)
    
    def forward(self, input):
        return self.weight.index_select(input)


class Tanh(Layer):
    def __init__(self):
        super().__init__()
    
    def forward(self, input):
        return input.tanh()


class Sigmoid(Layer):
    def __init__(self):
        super().__init__()
    
    def forward(self, input):
        return input.sigmoid()
    

class CrossEntropyLoss(object):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, input, target):
        return input.cross_entropy(target)

class MSELoss(object):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, input, target):
        dif = input - target
        return (dif * dif).sum(0)
    
class RNNCell(Layer):
    
    def __init__(self, n_inputs, n_hidden, n_output, activation='sigmoid'):
        super().__init__()

        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        if(activation == 'sigmoid'):
            self.activation = Sigmoid()
        elif(activation == 'tanh'):
            self.activation == Tanh()
        else:
            raise Exception("Non-linearity not found")

        self.w_ih = Linear(n_inputs, n_hidden)
        self.w_hh = Linear(n_hidden, n_hidden)
        self.w_ho = Linear(n_hidden, n_output)
        
        self.parameters += self.w_ih.get_parameters()
        self.parameters += self.w_hh.get_parameters()
        self.parameters += self.w_ho.get_parameters()        
    
    def forward(self, input, hidden):
        from_prev_hidden = self.w_hh.forward(hidden)
        combined = self.w_ih.forward(input) + from_prev_hidden
        new_hidden = self.activation.forward(combined)
        output = self.w_ho.forward(new_hidden)
        return output, new_hidden
    
    def init_hidden(self, batch_size=1):
        return Tensor(np.zeros((batch_size,self.n_hidden)), autograd=True)
    
class LSTMCell(Layer):
    
    def __init__(self, n_inputs, n_hidden, n_output):
        super().__init__()

        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output

        self.xf = Linear(n_inputs, n_hidden)
        self.xi = Linear(n_inputs, n_hidden)
        self.xo = Linear(n_inputs, n_hidden)        
        self.xc = Linear(n_inputs, n_hidden)        
        
        self.hf = Linear(n_hidden, n_hidden, bias=False)
        self.hi = Linear(n_hidden, n_hidden, bias=False)
        self.ho = Linear(n_hidden, n_hidden, bias=False)
        self.hc = Linear(n_hidden, n_hidden, bias=False)        
        
        self.w_ho = Linear(n_hidden, n_output, bias=False)
        
        self.parameters += self.xf.get_parameters()
        self.parameters += self.xi.get_parameters()
        self.parameters += self.xo.get_parameters()
        self.parameters += self.xc.get_parameters()

        self.parameters += self.hf.get_parameters()
        self.parameters += self.hi.get_parameters()        
        self.parameters += self.ho.get_parameters()        
        self.parameters += self.hc.get_parameters()                
        
        self.parameters += self.w_ho.get_parameters()        
    
    def forward(self, input, hidden):
        
        prev_hidden = hidden[0]        
        prev_cell = hidden[1]
        
        f = (self.xf.forward(input) + self.hf.forward(prev_hidden)).sigmoid()
        i = (self.xi.forward(input) + self.hi.forward(prev_hidden)).sigmoid()
        o = (self.xo.forward(input) + self.ho.forward(prev_hidden)).sigmoid()        
        g = (self.xc.forward(input) + self.hc.forward(prev_hidden)).tanh()        
        c = (f * prev_cell) + (i * g)

        h = o * c.tanh()
        
        output = self.w_ho.forward(h)
        return output, (h, c)
    
    def init_hidden(self, batch_size=1):
        init_hidden = Tensor(np.zeros((batch_size,self.n_hidden)), autograd=True)
        init_cell = Tensor(np.zeros((batch_size,self.n_hidden)), autograd=True)
        init_hidden.data[:,0] += 1
        init_cell.data[:,0] += 1
        return (init_hidden, init_cell)

## 学习检测垃圾邮件

In [32]:
spam_idx = to_indices(spam)
ham_idx = to_indices(ham)

train_spam_idx = spam_idx[:-1000]
train_ham_idx = ham_idx[:-1000]

test_spam_idx = spam_idx[-1000:]
test_ham_idx = ham_idx[-1000:]

train_data = list()
train_target = list()

test_data = list()
test_target = list()

for i in range(max(len(train_spam_idx), len(train_ham_idx))):
    train_data.append(train_spam_idx[i%len(train_spam_idx)])
    train_target.append([1])
    
    train_data.append(train_ham_idx[i%len(train_ham_idx)])
    train_target.append([0])
    
for i in range(max(len(test_spam_idx), len(test_ham_idx))):
    test_data.append(test_spam_idx[i%len(test_spam_idx)])
    test_target.append([1])
    
    test_data.append(test_ham_idx[i%len(test_ham_idx)])
    test_target.append([0])
    
def train(model, input_data, target_data, batch_size=100, iterations=5):
    n_batches = int(len(input_data) / batch_size)
    for iteration in range(iterations):
        iter_loss = 0
        for b_i in range(n_batches):
            (batch_start, batch_end) = (b_i * batch_size, (b_i+1) * batch_size)
            model.weight.data[word2index['unk']] *= 0
            input = Tensor(input_data[batch_start: batch_end], autograd=True)
            target = Tensor(target_data[batch_start: batch_end], autograd=True)
            
            pred = model.forward(input).sum(1).sigmoid()
            loss = criterion.forward(pred, target)
            loss.backward()
            optim.step()
            
            iter_loss += loss.data[0]
            sys.stdout.write('\r\tLoss:' + str(iter_loss / batch_end))
        print()
    return model

def test(model, test_input, test_output):
    model.weight.data[word2index['unk']] *= 0
    
    input = Tensor(test_input, autograd=True)
    target = Tensor(test_output, autograd=True)
    
    pred = model.forward(input).sum(1).sigmoid()
    return ((pred.data > 0.5) == target.data).mean()

In [33]:
model = Embedding(vocab_size=len(vocab), dim=1)
model.weight.data *= 0
criterion = MSELoss()
optim = SGD(parameters=model.get_parameters(), alpha=0.1)

for i in range(3):
    model = train(model, train_data, train_target, iterations=1)
    print('% Correct on Test Set:' + str(test(model, test_data, test_target) * 100))

	Loss:0.010722979357422575
% Correct on Test Set:99.75
	Loss:0.0021028410557890277
% Correct on Test Set:100.0
	Loss:0.0011595075974399477
% Correct on Test Set:100.0


## 联邦化 

In [34]:
bob = (train_data[0:1000], train_target[0:1000])
alice = (train_data[1000:2000], train_target[1000:2000])
sue = (train_data[2000:], train_target[2000:])

In [35]:
import copy
for i in range(3):
    print('Starting Training Round...')
    print('\tStep1: send the model to bob')
    bob_model = train(copy.deepcopy(model), bob[0], bob[1], iterations=1)
    
    print('\tStep2: send the model to alice')
    alice_model = train(copy.deepcopy(model), alice[0], alice[1], iterations=1)
    
    print('\tStep3: send the model to sue')
    sue_model = train(copy.deepcopy(model), sue[0], sue[1], iterations=1)
    
    print('\n\tAverage Everyone\'s New Models ')
    model.weight.data = (bob_model.weight.data + alice_model.weight.data + sue_model.weight.data) / 3
    print("\t% Correct on Test Set: " + \
          str(test(model, test_data, test_target)*100))
    print('\n Repeat!\n')
    

Starting Training Round...
	Step1: send the model to bob
	Loss:0.0011451959675176147
	Step2: send the model to alice
	Loss:0.0009465600605861596
	Step3: send the model to sue
	Loss:0.0009149338888900243

	Average Everyone's New Models 
	% Correct on Test Set: 100.0

 Repeat!

Starting Training Round...
	Step1: send the model to bob
	Loss:0.0011451959675176147
	Step2: send the model to alice
	Loss:0.0009465600605861594
	Step3: send the model to sue
	Loss:0.0009149338888900243

	Average Everyone's New Models 
	% Correct on Test Set: 100.0

 Repeat!

Starting Training Round...
	Step1: send the model to bob
	Loss:0.0011451959675176147
	Step2: send the model to alice
	Loss:0.0009465600605861594
	Step3: send the model to sue
	Loss:0.0009149338888900243

	Average Everyone's New Models 
	% Correct on Test Set: 100.0

 Repeat!



### 深入联邦学习
* 隐私问题——从权重更新得知数据的内容

In [36]:
import copy

bobs_email = ["my", "computer", "password", "is", "pizza"]

bob_input = np.array([[word2index[x] for x in bobs_email]])
bob_target = np.array([[0]])

# model = Embedding(vocab_size=len(vocab), dim=1)
# model.weight.data *= 0

bobs_model = train(copy.deepcopy(model), bob_input, bob_target, iterations=1, batch_size=1)

	Loss:0.08395573209431821


In [37]:
for i,v in enumerate(bob_model.weight.data - model.weight.data):
    if v is not 0:
        print(vocab[i])


mnzfg
hissproof
graywacke
sh
dobson
256
542862889
precocity
pluralism
exhaustion
compartment
pupil
zcthi
primitiva
automated
melbourne
anzus
135664
clobber
osiyemi
meoh
athietic
789355
molar
rawboned
mat
referance
mehrmals
transco
villareal
999
decile
teamlead
mre
burgeon
curiosity
circulatory
suppresses
ppkse
rxqlwxca
bkn
custody
creating
6521
ermndbs
eb
escapee
lateness
luke
electrocardiograph
bemused
ailments
wnw
quadratic
fission
gather
landry
trump
jaquet
ndwsmkrsvpjlfygao
besit
onyx
berke
residence
nameable
1848
property
professor
rachel
084
milienniums
dq
6992
looking
headhache
worry
noble
pat
xu
ardency
plg
lawsuit
mothers
edwardian
computed
gcgkmjcmeb
everett
dri
30599
bmc
waxy
vail
dedicate
dhttp
84079
stron
border
ssp
calphalon
jouk
chalmers
impressionism
vincentv
packets
sworn
infotainment
ofhappiness
supports
gram
betsey
knott
1465
prefab
tidinesses
windowpane
trent
fraser
thumbnail
bimini
cordoned
vhclytwra
zomtc
6707584
continued
visual
wardrobes
1150
irvine
episodes
tr

ghoulish
simplest
incl
wholiy
boazycefnbr
tueday
bfont
flung
pzmlwyj
deltanet
vetter
lhyic
mizl
712358
knidw
xpembt
assyrian
8233
da
underv
crook
pgrusm
shinteraysion
jlkloepfer
finley
burdensome
attorneyatlaw
barbican
oilshale
minutemen
redirects
ejaculation
summate
elevate
doctormto
rfuvrijmw
jackpot
economou
crying
wsrxtivm
405
yqpirvsj
dodecahedra
zinn
univ
horrify
escada
committeeman
coordinator
salinas
thebr
rental
fiori
tightest
societal
gambil
tusgar
sydney
trespass
tohcs
fixing
1660
feop
4810
mallak
drviv
fdtjji
scqcxx
donindfw
decide
hackneyed
silicon
breadroot
blondie
drgwn
curtailed
gigs
rrcotten
grayson
questioning
ballistic
gravitation
clearance
spells
loneyy
snd
prudency
trqklsdxbr
professionally
greif
beat
material
toccata
predictor
wwolytile
srv
glancey
cccccc
medg
cilhh
alfredo
tsoml
blithe
promptly
smith
arme
spadea
lively
saaaaalute
vernier
seeger
7720
fhcxw
cosby
returned
ertagacha
cle
eqtnm
379
wjones
revol
35759
tapis
infrastructure
simultaneous
partnered
aeneid


yuck
einem
dubuis
xaponax
misconjugate
devout
dp
maul
astute
idol
valerie
147
garvey
spokesman
idftkyjv
1384
fi
vdzqi
diversity
exempt
physiotherapy
attending
kenseaman
infotex
negatively
who
unsuccessful
247754
solitary
0989708
saucy
hhhkyfni
pkemper
omvtxcp
declamatory
east
carpenter
cerussite
referenence
margins
984959
alert
toronto
warmer
roil
juicy
patel
arachne
bidders
hillarious
spelled
goodbody
investissement
kellerman
1360
gulley
respecter
belcher
singletary
usoncology
databases
offers
895651
shaw
cybernetics
warwick
brajenovlc
rwjlxrqv
patentee
halt
canceled
miscellaneous
291
pietism
apologetic
qsne
mlluaryft
ordinate
pants
possiblities
4599
cold
701
tgcross
nirvanal
ozqexybc
pgkdqa
astronomy
notebookplus
dooley
arcsin
comatose
funerals
hewett
headstand
confidante
uyouv
sekpf
experts
woss
musty
thes
08
tofb
nuclide
proceeding
465490
gallishaw
jkxgden
tragedian
puacz
????????????
morelock
unblanched
voeiiu
scaie
252058
wrap
beaver
abraham
hvs
quarryman
waved
phosphor
proccess


replicated
heavens
memc
onlinewe
pagemaker
badges
endless
hamelb
slates
waived
epgqlahhkqqltj
42
sheppard
mary
malleable
deconvolution
5952
canteen
phillies
beltway
droopy
splashed
problem
cloth
9272
wits
dication
use
anise
hotties
shijpping
haujvzx
prescrlptlonneeded
decidedly
dobbin
7149
wtmym
fondren
welavo
beesn
copywriter
thurs
slackly
lability
udog
380578
ssurprised
metamorphose
modern
yjycunv
inelegant
crematory
rdqf
inexplicable
114096
guardia
flh
isp
hurst
mickey
465322
981384
later
constantinople
tier
zbbtgclu
laudanum
heig
878
groom
tenessee
oxhufzrhh
code
stephanie
triplett
forra
barlow
verrotst
madina
sorkina
poem
kjrztl
fbbsrs
utdallas
bezel
eting
colspan
propeller
compendia
cvonyy
toy
dzcjosm
astrology
cdkbrobwvthhrd
wfgnoud
asparagus
seldest
101574
kentuckyhereditary
piayers
fleck
counter
trisodium
casuality
tavist
oilcloth
752480
depressor
wmcobb
nvwzb
correio
erode
staffing
perplexed
michelle
expiore
tgp
fir
excavate
ulead
badminton
inapprolpriate
dew
my
rno
vonage
22

bnr
dis
rikiauho
xosloren
golden
flowers
strings
uppercase
sounds
mash
702
ludwig
123395
slot
w??t?ng
marx
partiaily
quartet
stones
yrl
lenart
headquarters
imperate
grade
whop
documentaries
amine
aycing
luxembourg
posse
440092
enroll
tuft
lang
mlxgizleh
unacceptably
girls
machinery
zykfe
064
budgeted
3518
napvn
mtv
hicabaw
willman
verne
kochind
pitman
bowed
questioned
dukepower
surplus
caloric
salaried
fbzvs
manger
maintian
executable
imaging
bon
debarwa
relaying
calc
mustang
incomeop
icgdkxu
uzvbtnznq
carver
charta
graceful
apps
ohgefi
6519
msnjoxwe
ubgez
lubell
fisher
wgr
min
lanna
494
cloudy
pomp
bunk
contracting
jlnyjxfp
452491
truong
grandeur
maseru
maldistribute
botello
congeal
testers
nemec
phon
softswitches
behest
linerpharmacy
mollie
47472
rooseveltian
nxfgr
cognitive
8830
deadwyler
sx
extranet
hertzog
hitch
platelet
patina
einfach
speedo
turmoil
australia
padding
wr
spekocials
mawkish
emcee
aquadrumanous
enjoying
casebook
radios
sikes
984486
prostitution
agmt
landon
609
snxfv

...

sulk
jaj
budapest
kratzer
jib
specialist
0980391
maxey
horace
0506
expatriates
greenwood
differences
calibration
allandl
constant
fountainmillion
teams
remiss
climbing
mitigated
unloaded
gtt
emulators
cops
eu
fashioend
zjwcqg
submissive
1774
tlucek
continuing
brenham
choosy
785
foliate
oat
scalar
immediate
1390
xprtung
didrex
arise
researched
stopband
scribe
1979
groote
triatomic
ggayle
rtxyj
rebelled
verifiable
spent
sale
rosy
colloquium
lbellamy
bird
vrsl
102056072
busines
bfi
honda
allegedly
selyn
assort
agrjhxb
debm
trivia
remoisve
hdr
tdtd
valderrama
generate
gutg
plans
woodlands
peabody
milestone
toni
straightedge
epstein
comingup
rapacious
conservation
rigorous
9736
embarrass
recyciing
flat
mush
pbmelz
tiber
disreagard
oibr
res
pick
frivolity
entities
baklava
rate
mvtm
acquainter
suppressant
inconvenience
virility
sterlinglenders
mocore
jkoutsi
dari
abed
poorer
ccmail
cult
mans
oqqrus
lpxcm
brochure
dutchmen
peeples
memoryricky
owaauvwpe
57065
wanogplhibr
shippi
educable
islamic

lover
777
returns
dies
limitations
tumbled
virology
evenbetter
proj
kne
2458
avionic
pyre
arrangeable
slav
phoenican
xml
glamorous
qicsxdq
bfg
telegeography
stemcaoiwz
2848
implementor
980432
genres
36777
whirligig
lettuce
vislt
procees
militate
241560
saloonkeep
confectionery
mutuel
derrek
dehydrate
hugo
epiphany
leah
embryology
791
extracted
525
enjoys
ads
??
tmbreeze
histochemic
windowxp
pray
wilsonilkj
fieu
mzol
jumpstart
randall
assaulting
registrar
abbey
10
upmtrmc
1106
finaiist
rt
athletes
708760
demarcate
winery
croquet
fishing
aching
ykrnvxzr
sfda
ostrander
texasreg
pecos
quirks
dukely
quackery
charlottesville
writers
qywdu
correspondance
definitively
cramer
chart
valiu
bilingual
axial
wofford
grinch
synergyog
shankster
138628
4745
russnjan
jna
advil
wakeup
etwas
universe
syngas
sportsmanship
sun
continua
picl
buttonagncm
ellenberger
yeah
tennessee
emotional
inst
exaggerate
pcc
quartz
reconnect
iiahby
9857
dehbi
ipkvoeuop
cacm
strata
212
fyhqdl
nijfejccv
steelmake
williamson
m

9124
mlrrwv
homecomings
upswing
aisdal
electricfool
cryogenic
33431
bitnet
jersey
pjthufppt
113858
haggle
extra
138049
03149
mcook
043
worlwide
preferences
hyyydroccodeeeine
operational
incessantly
arfxii
constructs
owpeft
keyshutdown
auntie
phentermiine
dede
aoe
repeated
joefunkconstr
depress
copied
diamonds
realtors
43142
intewrest
tsuoah
jetliner
4028
saturated
whichever
chipmunk
lwk
replications
frb
href
assault
pennies
6831
jlro
conf
boastful
depositor
money
nearby
collar
geoduck
elisha
txs
maintains
rabbi
kingbird
enforcible
entre
sateliite
idea
distortion
djilt
hodge
crinkle
checkbook
rolled
octol
atwjztyywl
rightful
consequently
oyla
koenig
yqloyfgwtfq
mzo
whittaker
jive
uunila
cockcrow
valdes
gefesselt
crutchfield
hidin
svga
fluorspar
scradford
conifer
neighbouring
amurphy
lastname
hiroko
shuyebtv
jayson
brice
osborn
educated
fldmi
roach
vcsc
kmid
epicure
alveolar
247676
uvdc
preston
cravings
lafollette
recapitalized
lack
homesick
diathermy
8608
perkins
originating
littie
de
t

compound
ymca
churchwoman
ggts
mus
clhwamilzn
dip
grandpa
amboynapointed
luckowski
emqheoj
yate
studied
corroteras
gcnmijxm
ccmekvq
tablebrtable
plcs
variations
howell
recieve
koreaherbal
influential
rowspan
smyooxddqwzif
hesitate
pleo
animations
practicably
987350
equipment
decline
sixteenth
caustic
litterbug
nmlve
oppose
nccygnus
concurring
gwenda
meniscus
xedkn
paperless
reveal
follette
ramping
documentation
buggy
bouncing
porpoise
caregiver
landolt
prank
ural
elena
zicqu
4001
greenland
tpwkkher
pjtccsh
shallow
walkthru
leech
ozonolysis
vl
cleavage
misieading
meinem
bie
kswkhm
ujnti
dass
donerb
explodes
conklin
missals
250
milieu
fees
986833
secondary
optima
greatest
nwizzsi
demurrer
534
osbourne
unvoted
secondhand
laud
fundamentally
bordeaux
aspecial
centrex
channel
hipa
effervescent
830689
reconverts
473545278
suicide
porcelain
schism
clarification
promoters
jbsdq
ough
bian
mossbunker
feldman
cockroachretroactive
ninfa
evergreen
mandy
ttnguyen
ncibynqrb
crosswalk
spending
vitrioli

merlin
negotiated
buckley
500
suck
6388
pvaovu
whelp
fatties
rjolly
milgram
marmalade
dauphine
haggerty
officia
40408
cadeau
galilee
goldberg
attire
affect
cyberbargain
healthy
gates
bite
9865
ostracism
january
377
pyrrhic
pai
cvoqklwbx
drafting
investigation
6009
annualiy
dukhuw
climax
fountain
developed
cotyledon
bjqo
modulation
eachother
heure
impunity
disagree
oquinn
zcytt
doarwv
nko
artprice
pentagon
937
reptile
intercourse
economic
brtoronto
doff
countess
380710
136
viltz
markhunt
columbic
oiwqeddqh
dgedeon
vlbllzymn
vonda
nnwehz
oboist
happyday
redwood
gbmxw
dpalmer
new
hostesses
blatzcleat
promenades
amra
shattuck
yall
sporter
strangest
gottfried
slum
thief
320
unionpacresinc
conserves
guqqmqal
reasor
jyr
wel
giqzfs
robyn
terceiro
armenta
382
magnanimity
hakemack
lucrative
andover
nosebleed
unearly
mouths
monument
worth
ambitious
bantuagenda
1000
malaise
outing
riggins
projecting
wmqfgj
edmondson
lgger
boeotia
1683
devi
fum
main
driiled
pulls
431644
dfxlqwdjz
wirde
etx
hierarch

## 安全聚合，同态加密

In [38]:
import phe

public_key, private_key = phe.generate_paillier_keypair(n_length=1024)

x = public_key.encrypt(5)

y = public_key.encrypt(3)

z = x + y

z_ = private_key.decrypt(z)
print("The Answer: " + str(z_))

The Answer: 8


In [39]:
public_key, private_key = phe.generate_paillier_keypair(n_length=128)

def train_and_encrypt(model, input, target, pubkey):
    new_model = train(copy.deepcopy(model), input, target, iterations=1)

    encrypted_weights = list()
    for val in new_model.weight.data[:,0]:
        encrypted_weights.append(public_key.encrypt(val))
    ew = np.array(encrypted_weights).reshape(new_model.weight.data.shape)
    
    return ew

In [40]:
for i in range(3):
    print("\nStarting Training Round...")
    print("\tStep 1: send the model to Bob")
    bob_encrypted_model = train_and_encrypt(copy.deepcopy(model), 
                                            bob[0], bob[1], public_key)

    print("\n\tStep 2: send the model to Alice")
    alice_encrypted_model = train_and_encrypt(copy.deepcopy(model), 
                                              alice[0], alice[1], public_key)

    print("\n\tStep 3: Send the model to Sue")
    sue_encrypted_model = train_and_encrypt(copy.deepcopy(model), 
                                            sue[0], sue[1], public_key)

    print("\n\tStep 4: Bob, Alice, and Sue send their")
    print("\tencrypted models to each other.")
    aggregated_model = bob_encrypted_model + \
                       alice_encrypted_model + \
                       sue_encrypted_model

    print("\n\tStep 5: only the aggregated model")
    print("\tis sent back to the model owner who")
    print("\t can decrypt it.")
    raw_values = list()
    for val in sue_encrypted_model.flatten():
        raw_values.append(private_key.decrypt(val))
    model.weight.data = np.array(raw_values).reshape(model.weight.data.shape)/3

    print("\t% Correct on Test Set: " + \
              str(test(model, test_data, test_target)*100))


Starting Training Round...
	Step 1: send the model to Bob
	Loss:0.0011451959675176147

	Step 2: send the model to Alice
	Loss:0.0009465600605861594

	Step 3: Send the model to Sue
	Loss:0.0009149338888900243

	Step 4: Bob, Alice, and Sue send their
	encrypted models to each other.

	Step 5: only the aggregated model
	is sent back to the model owner who
	 can decrypt it.
	% Correct on Test Set: 100.0

Starting Training Round...
	Step 1: send the model to Bob
	Loss:0.018781076276129308

	Step 2: send the model to Alice
	Loss:0.021959021980999564

	Step 3: Send the model to Sue
	Loss:0.020154686637448097

	Step 4: Bob, Alice, and Sue send their
	encrypted models to each other.

	Step 5: only the aggregated model
	is sent back to the model owner who
	 can decrypt it.
	% Correct on Test Set: 100.0

Starting Training Round...
	Step 1: send the model to Bob
	Loss:0.08585344314653713

	Step 2: send the model to Alice
	Loss:0.09230128994754407

	Step 3: Send the model to Sue
	Loss:0.0931310164